In [ ]:
class CheckpointLoaderCallback(TrainerCallback):
    def __init__(self):
      self.checkpoint_count = 0

    def on_save(self, args, state, control, **kwargs):
      self.checkpoint_count += 1 
      shutil.move(f"/content/checkpoint-{self.checkpoint_count*args.save_steps}.zip","/content/drive/MyDrive")
  

    


In [ ]:
authors = ['Ataol Behramoğlu', 'Attila İlhan', 'Cahit Sıtkı Tarancı', 'Cahit Zarifoğlu',
 'Can Yücel', 'Cemal Safi', 'Cemal Süreya', 'Edip Cansever',
 'Mehmet Akif Ersoy', 'Nazım Hikmet Ran', 'Necip Fazıl Kısakürek',
 'Orhan Veli Kanık', 'Turgut Uyar', 'Özdemir Asaf', 'Ümit Yaşar Oğuzcan']

In [ ]:

drive.mount('/content/drive')

In [ ]:
!pip install transformers
!pip install datasets


In [ ]:

from google.colab import drive
import shutil
import numpy as np
import pandas as pd
from IPython.display import clear_output
import transformers
from transformers import BertForSequenceClassification,BertTokenizer,Trainer,TrainingArguments,TrainerCallback
import torch
from datasets import Dataset
import ast

In [ ]:
tokenizer = BertTokenizer.from_pretrained("bert-base-multilingual-cased")
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels = len(authors))
checkpoint_loader = CheckpointLoaderCallback()
clear_output()

In [ ]:
train_dataset = pd.read_csv("/content/drive/MyDrive/train_dataset.csv", sep=";")
for column in ["input_ids","attention_mask"]:
    for i in range(0,len(train_dataset[column])):
        train_dataset[column][i] = np.array(ast.literal_eval(train_dataset[column][i]))

for i in range(0,len(train_dataset["label"])):
        train_dataset["label"][i] = np.array(train_dataset["label"][i])

clear_output()

In [ ]:
test_dataset = pd.read_csv("/content/drive/MyDrive/test_dataset.csv",sep=";")
for column in ["input_ids","attention_mask"]:
    for i in range(0,len(test_dataset[column])):
        test_dataset[column][i] = np.array(ast.literal_eval(test_dataset[column][i]))


for i in range(0,len(test_dataset["label"])):
    test_dataset["label"][i] = np.array(test_dataset["label"][i])

clear_output()

In [ ]:
train_dataset = Dataset.from_pandas(train_dataset)
test_dataset = Dataset.from_pandas(test_dataset)

In [ ]:
training_args = TrainingArguments(
     output_dir="test-trainer", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=5, # number of training epochs
    #learning_rate=0.00001,
    per_device_train_batch_size=8, # batch size for training
    per_device_eval_batch_size=8,  # batch size for evaluation
    eval_steps = 1000, # Number of update steps between two evaluations.
    logging_steps = 100,
    save_steps=1000, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer),
    callbacks = [checkpoint_loader]
)

In [ ]:
trainer.train()
clear_output()

In [ ]:
model.save_pretrained("/content/drive/MyDrive/last")

Configuration saved in /content/drive/MyDrive/last/config.json
Model weights saved in /content/drive/MyDrive/last/pytorch_model.bin
